 # 통계적 시계열 모델링

# 1.환경준비

## (1) 라이브러리 로딩

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as spst
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from sklearn.metrics import *
from sklearn.model_selection import train_test_split

import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore')
warnings.simplefilter('ignore', ConvergenceWarning)

## (2) 함수 생성

### 1) 결과 시각화

In [ ]:
def plot_model_result(y_train, y_val, pred) :
    pred = pd.Series(pred, index = y_val.index)

    # 전체 시각화
    plt.figure(figsize = (20,12))
    plt.subplot(2,1,1)
    plt.plot(y_train, label = 'train')
    plt.plot(y_val, label = 'val')
    plt.plot(pred, label = 'pred')
    plt.legend()
    plt.grid()

    plt.subplot(2,1,2)
    plt.plot(y_val, label = 'val')
    plt.plot(pred, label = 'pred')
    plt.legend()
    plt.grid()

    plt.show()

### 2) 잔차분석

In [ ]:
def residual_diag(residuals, lags = 30) :
    print('* 정규성 검정(> 0.05) : ', round(spst.shapiro(residuals)[1],5))
    print('* 정상성 검정(< 0.05) : ', round(sm.tsa.stattools.adfuller(residuals)[1],5))
    print('* 자기상관성 확인(ACF, PACF)')
    fig,ax = plt.subplots(1,2, figsize = (15,5))
    plot_acf(residuals, lags = lags, ax = ax[0])
    plot_pacf(residuals, lags = lags, ax = ax[1])
    plt.show()

## (3) 데이터 불러오기

In [ ]:
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/SeoulBike_Simple.csv'
bike = pd.read_csv(path)
bike['Datetime'] = pd.to_datetime(bike['Datetime'] )
bike.rename(columns={'Rented Bike Count':'Count'}, inplace = True)
bike = bike.loc[bike['Datetime'].between('2018-06-11','2018-08-13', inclusive = 'left'),
                ['Datetime', 'Temperature', 'Humidity','Count']]
bike.reset_index(drop = True, inplace = True)

## (4) 데이터 둘러보기

In [ ]:
bike.describe(include = 'all').T

In [ ]:
# 마지막 14일의 그래프를 그려 봅시다.
size = 24 * 14
temp = bike.iloc[-size:]
plt.figure(figsize = (20,8))
plt.plot('Datetime', 'Count', data = temp)
plt.grid()
plt.show()

# 2.기본 전처리

## (1) y 만들기

In [ ]:
data = bike.copy()

In [ ]:
data['y'] = data['Count'].shift(-2)

In [ ]:
data = data.iloc[:-2]

## (2) NaN 조치

In [ ]:
data.fillna(method = 'ffill', inplace = True)

## (3) 데이터 분할

### 1) x, y 나누기

In [ ]:
target = 'y'

x = data.drop([target, 'Datetime'], axis = 1) #제거할 때, date도 제거
y = data.loc[:, target]

### 2) train, val 분할
* 1회 분할 : train_test_split( x, y, test_size= , shuffle = False)
    * test_size : 소수 - 비율, 자연수 - 갯수
    * shuffle = False : 섞지 말고 데이터 끝에서 test_size 만큼 자르기

In [ ]:
val_size = 24 * 14
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = val_size, shuffle = False)

# 3.모델링1 : ARIMA

## (1) y 값 살펴보기

In [ ]:
residual_diag(y_train, lags = 50)

## (2) 모델링 : 초기모델

* p, d, q 값을 어떻게 정해야 할까요?
* AR의 p 차수와 MA q 차수 모두 값이 필요해 보입니다. 일단 1, 1을 지정합시다.
* d : 정상성을 띤 데이터가 아니므로 1로 지정

### 1) 학습

* sm.tsa.SARIMAX(train, order=(p,d,q)).fit()
    * 모델 선언시 train이 포함
    * .fit()으로 학습.

In [ ]:
# ARIMA 모델링
m1 = sm.tsa.SARIMAX(y_train, order=(1,1,1)).fit()

### 2) 평가

#### ① 잔차진단

* 모델.resid : 잔차를 뽑을 수 있습니다.
* 위에서 만든 함수 residual_diag 를 사용하여 잔차진단을 해 봅시다.

In [ ]:
residuals = m1.resid
residual_diag(residuals)

#### ② AIC
* 선형 모델에서의 적합도와, feature가 과도하게 늘어나는 것을 방지하도록 설계된 통계량이 AIC 입니다.
* 값이 작을 수록 좋은 모델
* 공식 : 𝐴𝐼𝐶=−2 ln⁡(𝐿)+2𝑘 ➡ - 모델의 적합도 + 변수의 갯수
* SARIMAX 모델.aic로 쉽게 통계량을 구할 수 있습니다.

In [ ]:
print('model1 AIC :', m1.aic)

#### ③ Validation

시계열 데이터로 실제값과 예측값에 대해 비교하여 그래프를 그려봅시다.

In [ ]:
pred = m1.forecast(size)
print('MAE :', mean_absolute_error(y_val, pred))
print('MAPE:', mean_absolute_percentage_error(y_val, pred))
print('R2  : ', r2_score(y_val, pred))

* 결과 시각화

In [ ]:
plot_model_result(y_train, y_val, pred)

## (3) 하이퍼파라미터 튜닝

* 실제로 p, d, q를 찾는 과정은 마치 Grid Search 처럼 값을 조금씩 조정해가며  최적의 모델을 찾아가는 과정과 유사합니다.
* 3 ~ 5분 정도 소요됩니다.


### 1) 학습

In [ ]:
from itertools import product

* 값의 범위 지정

In [ ]:
# product 함수를 이용하여 값의 조합을 구성
p = [2,3,4,5]
q = [2,3,4,5]
d = [1]
iter = list(product(p,d,q))
iter

* 튜닝

In [ ]:
# 반복문 진행율 확인
from tqdm import tqdm

In [ ]:
# 빈 리스트
mae, aic = [],[]

# 반복문으로 모델 생성하고 mae, aic 값 계산 후 저장
for i in tqdm(iter) : # tqdm!
    model_fit = sm.tsa.SARIMAX(y_train, order=(i[0], i[1], i[2])).fit()
    pred = model_fit.forecast(size)
    mae.append(mean_absolute_error(y_val, pred))
    aic.append(model_fit.aic)

* 튜닝 결과 확인

In [ ]:
# 가장 성능이 좋은 모델 조회
result = pd.DataFrame({'params(p,d,q)' : iter, 'mae' : mae, 'aic':aic})

display(result.loc[result['mae'] == result.mae.min()])
display(result.loc[result['aic'] == result.aic.min()])

In [ ]:
m1_1 = sm.tsa.SARIMAX(y_train, order=(2, 1, 4)).fit()

### 2) 평가

#### ① 잔차진단

* residual_diag

In [ ]:
residuals = m1_1.resid
residual_diag(residuals)

#### ② AIC
* 선형 모델에서의 적합도와, feature가 과도하게 늘어나는 것을 방지하도록 설계된 통계량이 AIC 입니다.
* 값이 작을 수록 좋은 모델
* 공식 : 𝐴𝐼𝐶=−2 ln⁡(𝐿)+2𝑘 ➡ - 모델의 적합도 + 변수의 갯수

In [ ]:
print('model AIC :', m1_1.aic)

#### ③ Validation

In [ ]:
pred = m1_1.forecast(size)
print('MAE :', mean_absolute_error(y_val, pred))
print('MAPE:', mean_absolute_percentage_error(y_val, pred))
print('R2  : ', r2_score(y_val, pred))

* 결과 시각화

In [ ]:
plot_model_result(y_train, y_val, pred)

# 4.모델링2 : SARIMA

## (1) 모델링 : 초기모델

### 1) 학습
* seasonality를 24로 두고 모델링을 수행해 봅시다.
* ARIMA 보다 시간이 더 걸립니다.

In [ ]:
# SARIMA 모델링(1~2분 소요)
m2 = sm.tsa.SARIMAX(y_train, order=(2,1,4), seasonal_order=(1,1,1,24)).fit()

### 2) 평가

#### ① 잔차진단

In [ ]:
residuals = m2.resid
residual_diag(residuals)

#### ② AIC

In [ ]:
print('model2_0 AIC :', m2.aic)

#### ③ Validation

In [ ]:
pred = m2.forecast(size)
print('MAE :', mean_absolute_error(y_val, pred))
print('MAPE:', mean_absolute_percentage_error(y_val, pred))
print('R2  : ', r2_score(y_val, pred))

* 결과 시각화

In [ ]:
plot_model_result(y_train, y_val, pred)

## (2) 하이퍼파라미터 튜닝
* 약 10분 수행.
* 범위를 늘리면 시간이 훨씬 증가합니다

### 1) 학습

In [ ]:
P = [1,2]
Q = [1,2]
D = [1]
mae, aic = [],[]
iter = list(product(P,D,Q))

for i in tqdm(iter) :
    model_fit = sm.tsa.SARIMAX(y_train, order=(2,1,4), seasonal_order=(i[0],i[1],i[2],24)).fit()
    pred = model_fit.forecast(size)
    mae.append( mean_absolute_error(y_val, pred))
    aic.append(model_fit.aic)

In [ ]:
result = pd.DataFrame({'params(P,D,Q)' : iter, 'mae' : mae, 'aic':aic})

result

In [ ]:
display(result.loc[result['mae'] == result.mae.min()])
display(result.loc[result['aic'] == result.aic.min()])

In [ ]:
m2_1 = sm.tsa.SARIMAX(y_train, order=(2,1,4), seasonal_order=(2,1,1,24)).fit()

### 2) 평가

#### ① 잔차진단

* residual_diag

In [ ]:
residuals = m2_1.resid
residual_diag(residuals)

#### ② AIC
* 선형 모델에서의 적합도와, feature가 과도하게 늘어나는 것을 방지하도록 설계된 통계량이 AIC 입니다.
* 값이 작을 수록 좋은 모델
* 공식 : 𝐴𝐼𝐶=−2 ln⁡(𝐿)+2𝑘 ➡ - 모델의 적합도 + 변수의 갯수

In [ ]:
print('model2_1 AIC :', m2_1.aic)

#### ③ Validation

In [ ]:
pred = m2_1.forecast(size)
print('MAE :', mean_absolute_error(y_val, pred))
print('MAPE:', mean_absolute_percentage_error(y_val, pred))
print('R2  : ', r2_score(y_val, pred))

* 결과 시각화

In [ ]:
plot_model_result(y_train, y_val, pred)

# 5.모델링3 : SARIMAX

## (1) 전처리

* 요일 : feature중 요일을 추가합니다.
* 활동시간 구분 : 시간을 추출하고, 다음과 같이 범주로 생성합니다.
    * 0 : 활동 없음, 1 : 일상 활동시간, 2 : 출퇴근
    * 0~6 : 0
    * 7~8 : 2
    * 9~16: 1
    * 17~20 : 2
    * 21~23 : 0



In [ ]:
data1 = data.copy()

* 요일

In [ ]:
data1['Weekday'] = data1['Datetime'].dt.day_name()

data1.head()

* 활동시간 구분


In [ ]:
data1['Active'] = pd.cut(data1['Datetime'].dt.hour, bins=[0,6,8,16,20,23], labels = [0,2,1,2,0], include_lowest = True)

In [ ]:
# 가변수화
cat_cols = ['Weekday','Active']
data1 = pd.get_dummies(data1, columns = cat_cols, drop_first = True)
data1.head()

In [ ]:
# 데이터 분할
target = 'y'
x = data1.drop([target, 'Datetime'], axis = 1) #제거할 때, date도 제거
y = data1.loc[:, target]

val_size = 24 * 14
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = val_size, shuffle = False)

## (2) 모델링
* 좀 더 걸립니다.(5분 이상..ㅜㅜ)

### 1) 학습

In [ ]:
m3_1 = sm.tsa.SARIMAX(y_train, order=(2,1,4), seasonal_order=(2,1,1,24), exog=x_train).fit()

### 2) 평가

#### ① 잔차진단

* residual_diag

In [ ]:
residuals = m3_1.resid
residual_diag(residuals)

#### ② AIC

In [ ]:
print('m3_1 AIC :', m3_1.aic)

#### ③ Validation
SARIMAX 모델을 생성하고, 예측할 때는 exog=x_val 옵션이 들어가야 함.

In [ ]:
pred = m3_1.forecast(size,  exog=x_val)

print('MAE :', mean_absolute_error(y_val, pred))
print('MAPE:', mean_absolute_percentage_error(y_val, pred))
print('R2  : ', r2_score(y_val, pred))

* 결과 시각화

In [ ]:
plot_model_result(y_train, y_val, pred)